# GA Burt Jones Bill Sponsorship

## Setup

In [6]:
import os, re, zipfile
from pathlib import Path
from glob import glob

# Directory that contains all your session zip files
ZIP_DIR = Path("zippedInputDataGA")

# Automatically collect all .zip files in that directory
ZIP_PATHS = sorted(glob(str(ZIP_DIR / "*.zip")))

OUT_DIR = Path("unzipped_with_id_GA")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def extract_id_from_zipname(zip_path: str) -> str:
    base = os.path.basename(zip_path)
    m_state = re.match(r"^([A-Z]{2})_", base)
    state = m_state.group(1) if m_state else "XX"
    m_sess = re.search(r"(\d{4})[-_](\d{4})", base)
    session = f"{m_sess.group(1)}-{m_sess.group(2)}" if m_sess else "unknown"
    return f"{state}-{session}"

created = []

for zip_path in ZIP_PATHS:
    name_id = extract_id_from_zipname(zip_path)
    with zipfile.ZipFile(zip_path) as zf:
        for member in zf.infolist():
            if member.is_dir():
                continue
            if not member.filename.lower().endswith(".csv"):
                continue
            orig_base = os.path.basename(member.filename)
            stem, ext = os.path.splitext(orig_base)
            new_name = f"{stem}__{name_id}{ext}"
            target_path = OUT_DIR / new_name
            with zf.open(member, "r") as src, open(target_path, "wb") as dst:
                dst.write(src.read())
            created.append(str(target_path))

print("Found ZIPs:", ZIP_PATHS)
print("Created files:")
for f in created:
    print(" -", f)


Found ZIPs: ['zippedInputDataGA\\GA_2013-2014_Regular_Session_CSV_20160101_0dde8c244bd91d036565ea4ae614f22e.zip', 'zippedInputDataGA\\GA_2015-2016_Regular_Session_CSV_20190101_3b89236cd9df642916571bd04751fba8.zip', 'zippedInputDataGA\\GA_2017-2018_Regular_Session_CSV_20200101_c12e6cf11c6c5a100ba4b33dadd20721.zip', 'zippedInputDataGA\\GA_2019-2020_Regular_Session_CSV_20210701_2cfb4e065757064475a6f1cc5525d5bc.zip', 'zippedInputDataGA\\GA_2021-2022_Regular_Session_CSV_20220513_7405a3a7cb4cd81f020d33720e289aad.zip', 'zippedInputDataGA\\GA_2023-2024_Regular_Session_CSV_20240507_f04063c97c50d2a6cd165266ed9c64db.zip', 'zippedInputDataGA\\GA_2025-2026_Regular_Session_CSV_20250514_1bfe8db1f47059fc8a4961fdeb5c2370.zip']
Created files:
 - unzipped_with_id_GA\bills__GA-2013-2014.csv
 - unzipped_with_id_GA\history__GA-2013-2014.csv
 - unzipped_with_id_GA\sponsors__GA-2013-2014.csv
 - unzipped_with_id_GA\rollcalls__GA-2013-2014.csv
 - unzipped_with_id_GA\votes__GA-2013-2014.csv
 - unzipped_with_id_G

## Burt Jones Cosponsored and Sponsored bills

In [7]:
# Build a session-by-session table of bills sponsored by a given person_id,
# with counts by status codes: 2 (Passed One Chamber), 4 (Signed By Gov), 5 (Vetoed).
# Uses the already-unzipped files in step1_unzipped_with_id/<table>__<SESSION>.csv
#
# Output: /mnt/data/sponsored_bills_summary_15158.csv

import pandas as pd
import re
from pathlib import Path

UNZIPPED_DIR = Path("unzipped_with_id_GA")
PERSON_ID = 15158

OUT_BASE = Path("burt_jones_bill_sponsorship_GA")
OUT_BASE.mkdir(parents=True, exist_ok=True)

# Discover sessions from sponsors__*.csv (ensures we're looking at sessions with sponsorship data)
sessions = sorted([re.match(r"sponsors__(.+)\.csv$", p.name).group(1)
                   for p in UNZIPPED_DIR.glob("sponsors__*.csv")
                   if re.match(r"sponsors__(.+)\.csv$", p.name)])

rows = []
for sid in sessions:
    sponsors_p = UNZIPPED_DIR / f"sponsors__{sid}.csv"
    bills_p = UNZIPPED_DIR / f"bills__{sid}.csv"
    if not sponsors_p.exists() or not bills_p.exists():
        continue

    sponsors = pd.read_csv(sponsors_p)
    bills = pd.read_csv(bills_p)

    # Ensure expected columns exist
    # LegiScan sponsors typically: bill_id, people_id, type, seq, position, sponsor_order, ... (varies)
    if "people_id" not in sponsors.columns:
        # Try alternative casing just in case
        cand = [c for c in sponsors.columns if c.lower() == "people_id"]
        if cand:
            sponsors.rename(columns={cand[0]: "people_id"}, inplace=True)

    if "bill_id" not in sponsors.columns and "bill" in sponsors.columns:
        sponsors.rename(columns={"bill": "bill_id"}, inplace=True)

    if "bill_id" not in bills.columns and "bill" in bills.columns:
        bills.rename(columns={"bill": "bill_id"}, inplace=True)

    # Filter sponsors by person
    sp = sponsors[sponsors["people_id"] == PERSON_ID].copy()
    if sp.empty:
        rows.append({
            "Session": sid,
            "Bills Sponsored (All Status #s)": 0,
            "Passed One Chamber (Status #2)": 0,
            "Signed By Gov (Status #4)": 0,
            "Vetoed (Status #5)": 0
        })
        continue

    bill_ids = sp["bill_id"].dropna().unique().tolist()
    b = bills[bills["bill_id"].isin(bill_ids)].copy()

    # Normalize status column name
    status_col = None
    for candidate in ["status", "status_id", "status_code"]:
        if candidate in b.columns:
            status_col = candidate
            break
    if status_col is None:
        # If no status column, count totals but set others to 0
        rows.append({
            "Session": sid,
            "Bills Sponsored (All Status #s)": len(bill_ids),
            "Passed One Chamber (Status #2)": 0,
            "Signed By Gov (Status #4)": 0,
            "Vetoed (Status #5)": 0
        })
        continue

    # Coerce to numeric if needed
    b[status_col] = pd.to_numeric(b[status_col], errors="coerce")

    total = len(b["bill_id"].unique())
    passed_one = int((b[status_col] == 2).sum())
    signed = int((b[status_col] == 4).sum())
    vetoed = int((b[status_col] == 5).sum())

    rows.append({
        "Session": sid,
        "Bills Sponsored (All Status #s)": total,
        "Passed One Chamber (Status #2)": passed_one,
        "Signed By Gov (Status #4)": signed,
        "Vetoed (Status #5)": vetoed
    })

summary = pd.DataFrame(rows).sort_values("Session").reset_index(drop=True)

combined_path = OUT_BASE / "burt_jones_GA_sponsored_bills_summary.csv"
summary.to_csv(combined_path, index=False)

str(out_path)

'sponsored_bills_summary_15158.csv'

## Burt Jones Sponsored Bills

In [9]:
import pandas as pd
import re
from pathlib import Path

UNZIPPED_DIR = Path("unzipped_with_id_GA")
PERSON_ID = 15158

# Discover sessions
sessions = sorted([re.match(r"sponsors__(.+)\.csv$", p.name).group(1)
                   for p in UNZIPPED_DIR.glob("sponsors__*.csv")
                   if re.match(r"sponsors__(.+)\.csv$", p.name)])

rows = []
for sid in sessions:
    sponsors_p = UNZIPPED_DIR / f"sponsors__{sid}.csv"
    bills_p = UNZIPPED_DIR / f"bills__{sid}.csv"
    if not sponsors_p.exists() or not bills_p.exists():
        continue

    sponsors = pd.read_csv(sponsors_p)
    bills = pd.read_csv(bills_p)

    # Normalize columns
    if "people_id" not in sponsors.columns:
        cand = [c for c in sponsors.columns if c.lower() == "people_id"]
        if cand:
            sponsors.rename(columns={cand[0]: "people_id"}, inplace=True)
    if "bill_id" not in sponsors.columns and "bill" in sponsors.columns:
        sponsors.rename(columns={"bill": "bill_id"}, inplace=True)
    if "bill_id" not in bills.columns and "bill" in bills.columns:
        bills.rename(columns={"bill": "bill_id"}, inplace=True)

    # Filter: only person_id==15158 and position==1 (primary sponsor)
    sp = sponsors[
        (sponsors["people_id"] == PERSON_ID) &
        (sponsors.get("position", 0) == 1)
    ].copy()

    if sp.empty:
        rows.append({
            "Session": sid,
            "Bills Sponsored (All Status #s)": 0,
            "Passed One Chamber (Status #2)": 0,
            "Signed By Gov (Status #4)": 0,
            "Vetoed (Status #5)": 0
        })
        continue

    bill_ids = sp["bill_id"].dropna().unique().tolist()
    b = bills[bills["bill_id"].isin(bill_ids)].copy()

    # Find status column
    status_col = None
    for candidate in ["status", "status_id", "status_code"]:
        if candidate in b.columns:
            status_col = candidate
            break

    if status_col is None:
        rows.append({
            "Session": sid,
            "Bills Sponsored (All Status #s)": len(bill_ids),
            "Passed One Chamber (Status #2)": 0,
            "Signed By Gov (Status #4)": 0,
            "Vetoed (Status #5)": 0
        })
        continue

    b[status_col] = pd.to_numeric(b[status_col], errors="coerce")

    total = len(b["bill_id"].unique())
    passed_one = int((b[status_col] == 2).sum())
    signed = int((b[status_col] == 4).sum())
    vetoed = int((b[status_col] == 5).sum())

    rows.append({
        "Session": sid,
        "Bills Sponsored (All Status #s)": total,
        "Passed One Chamber (Status #2)": passed_one,
        "Signed By Gov (Status #4)": signed,
        "Vetoed (Status #5)": vetoed
    })

summary = pd.DataFrame(rows).sort_values("Session").reset_index(drop=True)

combined_path = OUT_BASE / "burt_jones_GA_sponsored_bills_summary_primary.csv"
summary.to_csv(combined_path, index=False)

print("Saved:", combined_path)


Saved: sponsored_bills_summary_15158.csv


## Burt Jones Sponsored Bills Tacker

In [21]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

def build_sponsorship_table(
    people_id: int,
    unzipped_dir: Path,
    require_primary_only: bool = False,  # set True if you only want position==1
) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
      Session, Bill Number, Date of introduction, Sponsorship Type,
      Status Description, Description, State Link

    Date of introduction:
      pulled from history__<session>.csv where bill_id matches and sequence == 1
    """

    # discover sessions from bills files present
    sessions = sorted({
        re.match(r"bills__(.+)\.csv$", p.name).group(1)
        for p in unzipped_dir.glob("bills__*.csv")
        if re.match(r"bills__(.+)\.csv$", p.name)
    })

    rows = []
    for sid in sessions:
        bills_p    = unzipped_dir / f"bills__{sid}.csv"
        sponsors_p = unzipped_dir / f"sponsors__{sid}.csv"
        history_p  = unzipped_dir / f"history__{sid}.csv"

        # need all three to produce the table per your new rule
        if not (bills_p.exists() and sponsors_p.exists() and history_p.exists()):
            continue

        bills    = pd.read_csv(bills_p)
        sponsors = pd.read_csv(sponsors_p)
        history  = pd.read_csv(history_p)

        # normalize common variants
        if "bill_id" not in sponsors.columns and "bill" in sponsors.columns:
            sponsors = sponsors.rename(columns={"bill": "bill_id"})
        if "bill_id" not in bills.columns and "bill" in bills.columns:
            bills = bills.rename(columns={"bill": "bill_id"})
        if "people_id" not in sponsors.columns:
            alt = [c for c in sponsors.columns if c.lower() == "people_id"]
            if alt:
                sponsors = sponsors.rename(columns={alt[0]: "people_id"})

        # filter to this person (and optionally only primary)
        sp = sponsors[sponsors["people_id"] == people_id].copy()
        if require_primary_only and "position" in sp.columns:
            sp = sp[sp["position"] == 1]
        if sp.empty:
            continue

        # Sponsor vs Cosponsor
        if "position" not in sp.columns:
            sp["position"] = np.nan
        sp["Sponsorship Type"] = np.where(sp["position"] == 1, "Sponsor", "Cosponsor")

        # --- Intro date from history: sequence == 1 ---
        # make sure sequence is numeric (some dumps store as text)
        if "sequence" in history.columns:
            history["sequence"] = pd.to_numeric(history["sequence"], errors="coerce")
            intro_map = (
                history[history["sequence"] == 1.0]
                .loc[:, ["bill_id", "date"]]
                .dropna(subset=["bill_id"])
                .rename(columns={"date": "Date of introduction"})
                .drop_duplicates(subset=["bill_id"], keep="first")
            )
        else:
            # if there is no 'sequence', fallback to earliest row per bill
            intro_map = (
                history
                .sort_values(["bill_id", "date"])
                .groupby("bill_id", as_index=False)["date"]
                .first()
                .rename(columns={"date": "Date of introduction"})
            )

        # Merge sponsorships -> bills -> intro date
        merged = (
            sp.merge(bills, on="bill_id", how="left", suffixes=("", "_bill"))
              .merge(intro_map, on="bill_id", how="left")
        )

        out = pd.DataFrame({
            "Session": sid,
            "Bill Number": merged.get("bill_number"),
            "Date of introduction": merged.get("Date of introduction"),
            "Sponsorship Type": merged.get("Sponsorship Type"),
            "Status Description": merged.get("status_desc", merged.get("status")),
            "Bill Description": merged.get("description"),
            "State Link": merged.get("state_link"),
        })

        rows.append(out)

    if rows:
        return pd.concat(rows, ignore_index=True)
    else:
        return pd.DataFrame(
            columns=[
                "Session","Bill Number","Date of introduction",
                "Sponsorship Type","Status Description","Bill Description","State Link"
            ]
        )
        
# Example run:
df = build_sponsorship_table(people_id=15158, unzipped_dir=UNZIPPED_DIR)
combined_path = OUT_BASE / "sponsored_cosponsored_15158.csv"
df.to_csv(combined_path, index=False)